In [196]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

### Defining Setup Time and Hourly Yield Constants

In [197]:
T = pd.read_csv('setup_constants_q5.csv')
R = pd.read_csv('yield_constants.csv')

In [198]:
T.index += 1
R.index += 1

### Defining Weekly Demand Constants

In [199]:
D = pd.read_csv('weekly_demands.csv')

In [200]:
D.index += 1

# Defining Problem

In [201]:
model = gp.Model('base_problem')

## Variables

### Adding Binary Setup Variables

In [202]:
setup_complete_tuples = [(0,0),(1,1),(2,3),(3,4),(4,2)]

In [203]:
# (1,2) in setup_complete_tuples

In [204]:
S = []
for i in range(5):
    S_i = []
    for j in range(5):
        S_i.append(model.addVar(vtype=GRB.BINARY, name=f"S_{i}_{j}", lb=0))
        if (i,j) in setup_complete_tuples:
            S_i[j].Start = 1
    S.append(S_i)

### Adding Binary Overtime Setup Variables

In [205]:
Sp = []
for i in range(5):
    Sp_i = []
    for j in range(5):
        Sp_i.append(model.addVar(vtype=GRB.BINARY, name=f"Sp_{i}_{j}", lb=0))
    Sp.append(Sp_i)

### Adding Production Variables

In [206]:
P = []
for i in range(5):
    P_i = []
    for j in range(5):
        P_i.append(model.addVar(vtype=GRB.CONTINUOUS, name=f"P_{i}_{j}", lb=0))
    P.append(P_i)

### Adding Overtime Production Variables

In [207]:
Pp = []
for i in range(5):
    Pp_i = []
    for j in range(5):
        Pp_i.append(model.addVar(vtype=GRB.CONTINUOUS, name=f"Pp_{i}_{j}", lb=0))
    Pp.append(Pp_i)

### Adding Use Variables

In [208]:
U = []
for i in range(5):
    U_i = []
    for j in range(5):
        U_i.append(model.addVar(vtype=GRB.BINARY, name=f"U_{i}_{j}", lb=0))
    U.append(U_i)

### Adding Overtime Use Variables

In [209]:
Up = []
for i in range(5):
    Up_i = []
    for j in range(5):
        Up_i.append(model.addVar(vtype=GRB.BINARY, name=f"Up_{i}_{j}", lb=0))
    Up.append(U_i)

## Constraints

### Demand Constraints

In [210]:
for i in range(5):
    constr_expr = gp.quicksum(P[j][i]*R.iloc[j,i] + Pp[j][i]*R.iloc[j,i] for j in range(5))
    model.addConstr(constr_expr >= D.iloc[0][i], name=f'Demand_Prod_{i+1}')

/var/folders/w_/z02fg3l56_jfc9tys40hqtdm0000gn/T/ipykernel_15742/2297407209.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  model.addConstr(constr_expr >= D.iloc[0][i], name=f'Demand_Prod_{i+1}')


### Normal Production Time Constraints

In [211]:
for i in range(5):
    constr_expr = gp.quicksum(S[i][j]*T.iloc[i,j] + P[i][j] for j in range(5))
    model.addConstr(constr_expr <= 120, name=f'Prod_Hrs_Mach_{i+1}')

### Overtime Production Time Constraints

In [212]:
for i in range(5):
    constr_expr = gp.quicksum(Sp[i][j]*T.iloc[i,j] + Pp[i][j] for j in range(5))
    model.addConstr(constr_expr <= 48, name=f'OT_Prod_Hrs_Mach_{i+1}')

### Production Use Linkage Constants
- Regular Time and Overtime are both included here

In [213]:
for i in range(5):
    for j in range(5):
        model.addConstr(P[i][j]*R.iloc[i,j] <= 10000*U[i][j], name=f'UseProdLinkage_Mach_{i+1}_Prod_{j+1}')
        model.addConstr(Pp[i][j]*R.iloc[i,j] <= 10000*(U[i][j]+Up[i][j]), name=f'OTUseProdLinkage_Mach_{i+1}_Prod_{j+1}')

### Setup Use Linkage Constants
- Regular Time and Overtime are both included here

In [214]:
for i in range(5):
    for j in range(5):
        model.addConstr(U[i][j] <= 10*S[i][j], name=f'SetupUseLinkage_Mach_{i+1}_Prod_{j+1}')
        model.addConstr(Up[i][j] <= 10*(S[i][j]+Sp[i][j]), name=f'SetupUseLinkage_Mach_{i+1}_Prod_{j+1}')

## Objective Function

In [215]:
obj_parts = []
for i in range(5):
    obj_parts.append(gp.quicksum(Sp[i][j]*T.iloc[i,j] + Pp[i][j] for j in range(5)))

In [216]:
model.setObjective(gp.quicksum(obj_parts),GRB.MINIMIZE)

In [217]:
model.update()

In [218]:
model.getObjective()

<gurobi.LinExpr: 8.0 Sp_0_3 + 10.0 Sp_1_0 + 10.0 Sp_2_1 + 8.0 Sp_3_1 + 20.0 Sp_4_4 + Pp_0_0 + Pp_0_1 + Pp_0_2 + Pp_0_3 + Pp_0_4 + Pp_1_0 + Pp_1_1 + Pp_1_2 + Pp_1_3 + Pp_1_4 + Pp_2_0 + Pp_2_1 + Pp_2_2 + Pp_2_3 + Pp_2_4 + Pp_3_0 + Pp_3_1 + Pp_3_2 + Pp_3_3 + Pp_3_4 + Pp_4_0 + Pp_4_1 + Pp_4_2 + Pp_4_3 + Pp_4_4>

In [219]:
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.0.0 24A348)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 115 rows, 150 columns and 295 nonzeros
Model fingerprint: 0x74992c0b
Variable types: 50 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 4e+03]

User MIP start produced solution with objective 91.0767 (0.00s)
Loaded user MIP start with objective 91.0767

Presolve removed 101 rows and 132 columns
Presolve time: 0.00s
Presolved: 14 rows, 18 columns, 44 nonzeros
Variable types: 13 continuous, 5 integer (5 binary)

Root relaxation: objective 8.984605e+01, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node

In [220]:
model.Status == GRB.OPTIMAL

True

In [221]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print(int(abs(S[i][j].X*T.iloc[i,j])), end=" ")
    print("")

Machine 1: 0 0 0 8 0 
Machine 2: 10 0 0 0 0 
Machine 3: 0 10 0 0 0 
Machine 4: 0 8 0 0 0 
Machine 5: 0 0 0 0 20 


In [222]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print(int(abs(Sp[i][j].X*T.iloc[i,j])), end=" ")
    print("")

Machine 1: 0 0 0 0 0 
Machine 2: 0 0 0 0 0 
Machine 3: 0 0 0 0 0 
Machine 4: 0 0 0 0 0 
Machine 5: 0 0 0 0 0 


In [223]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print((abs(P[i][j].X)), end=" ")
    print("")

Machine 1: 49.583333333333336 0.0 0.0 62.416666666666664 0.0 
Machine 2: 110.0 0.0 0.0 0.0 0.0 
Machine 3: 0.0 110.0 0.0 0.0 0.0 
Machine 4: 0.0 53.333333333333336 58.666666666666664 0.0 0.0 
Machine 5: 0.0 0.0 0.0 40.1474358974359 59.8525641025641 


In [224]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print((abs(Pp[i][j].X)), end=" ")
    print("")

Machine 1: 0.0 0.0 0.0 0.0 0.0 
Machine 2: 0.0 0.0 0.0 0.0 0.0 
Machine 3: 0.0 9.595959595959606 0.0 0.0 0.0 
Machine 4: 0.0 0.0 48.0 0.0 0.0 
Machine 5: 0.0 0.0 0.0 0.0 33.48076923076923 


In [225]:
for i in range(5):
    print(f"Part {i+1}:", round(sum([abs(P[j][i].X+Pp[j][i].X)*R.iloc[j,i] for j in range(5)])), end=" ")
    print("")

Part 1: 3500 
Part 2: 3000 
Part 3: 4000 
Part 4: 4000 
Part 5: 2800 


In [226]:
print('Final Objective Value: ', model.ObjVal)

Final Objective Value:  91.07672882672884
